## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-05-17-21 +0000,nypost,Comedian Reggie Carroll shot dead at 52 in Mis...,https://nypost.com/2025/08/25/us-news/comedian...
1,2025-08-25-05-03-53 +0000,bbc,"Get ready for fracking, Reform UK tells energy...",https://www.bbc.com/news/articles/c74172wlezwo...
2,2025-08-25-04-48-47 +0000,startribune,Review: Indigo Girls and Melissa Etheridge mak...,https://www.startribune.com/melissa-etheridge-...
3,2025-08-25-04-34-35 +0000,nyt,Monday Briefing: Would Ukraine Give Up the Don...,https://www.nytimes.com/2025/08/25/briefing/uk...
4,2025-08-25-04-27-32 +0000,nyt,This Is the End of China’s Once Mightiest Prop...,https://www.nytimes.com/2025/08/24/business/ch...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
242,trump,19
50,new,13
29,ukraine,12
361,russia,9
458,war,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
95,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...,59
126,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...,54
115,2025-08-24-14-39-41 +0000,nypost,Trump announces he’ll attend Ryder Cup in New ...,https://nypost.com/2025/08/24/us-news/trump-an...,54
94,2025-08-24-18-08-34 +0000,nypost,Vance reveals what Putin has conceded in Ukrai...,https://nypost.com/2025/08/24/us-news/vance-re...,50
147,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...,49


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
95,59,2025-08-24-18-08-34 +0000,nypost,Russia’s top diplomat says Zelensky can’t sign...,https://nypost.com/2025/08/24/world-news/russi...
147,49,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...
91,33,2025-08-24-18-30-00 +0000,nypost,Australian mom accused of killing American ste...,https://nypost.com/2025/08/24/world-news/lisa-...
126,27,2025-08-24-14-05-19 +0000,nypost,Russia’s foreign minister throws up a new cond...,https://nypost.com/2025/08/24/us-news/russias-...
107,24,2025-08-24-16-34-05 +0000,nypost,Rep. Lawler backs Trump sending National Guard...,https://nypost.com/2025/08/24/us-news/rep-lawl...
19,23,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
22,23,2025-08-25-01-35-57 +0000,nypost,Freak hot tub incident almost kills 2 elderly ...,https://nypost.com/2025/08/24/us-news/freak-ho...
56,22,2025-08-24-22-16-36 +0000,nypost,How scion of one of Canada’s richest families ...,https://nypost.com/2025/08/24/us-news/how-thom...
143,22,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
70,21,2025-08-24-20-51-10 +0000,nypost,Israeli airstrikes hit Yemen’s capital after H...,https://nypost.com/2025/08/24/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
